# Generate tables and other tabular data for DMD FJLT

In [ ]:
import matplotlib as mpl
from matplotlib.ticker import MaxNLocator
import scipy.signal as sig
import numpy as np
from tqdm import tqdm
import sys
sys.path.append('../../../utils')
import gc
from TurboFJLT import *
from InstantFrequency import inst_freq
from TurboFJLT_helpers import FJLT, TurboHDF5Reader, TurboVisual

In [ ]:
%config InlineBackend.figure_format='retina'


In [ ]:
def jfm_table(headings, data_arr):
    assert len(headings) == data_arr.shape[1]
    num_cols = len(headings)

    # Convert the data into the correct form
    head_str = " & ".join(map(str, headings)) + " \\\\"
    arr_str = " \\\\\n".join([" & ".join(map(str, row)) for row in data_arr])

    table_structure = (
        r"\begin{table}",
        r"\begin{center}",
        r"\def~{\hphantom{0}}",
        r"\begin{tabular}" + "{{{}}}".format("r" + "r"*(num_cols-1)),
        "{}".format(head_str),
        "{}".format(arr_str),
        r"\end{tabular}",
        r"\caption{EMPTY CAPTION}",
        r"\label{tab:empty_label}",
        r"\end{center}",
        r"\end{table}")

    for elem in table_structure:
        print(elem)

In [ ]:
def fjlt_helper(reader, num_linking_snapshots):
    fjlt = FJLT(reader.state_dim, num_linking_snapshots, max_distortion=0.01)
    compression_rel_state = reader.state_dim/fjlt.embedding_dim
    compression_rel_hankel = reader.state_dim*fjlt.snapshots_dim/fjlt.embedding_dim

    table_row = [
        "${:2d}$".format(fjlt.snapshots_dim),
        "${:7,d}$".format(fjlt.embedding_dim).replace(',', '\,'),
        "${:4.1f}$".format(compression_rel_state),
        "${:5.1f}$".format(compression_rel_hankel)
    ]

    headings = [
        r"$n_{sp}$",
        r"$k$",
        r"$n_{dof}/{k}$",
        r"$n_{dof}n_{sp}/{k}$"
    ]

    return table_row, headings

In [ ]:
datafile = "../../../data/fine_airfoil_cascade.h5"
reader = TurboHDF5Reader(datafile)
print(reader)

In [ ]:
num_linking_snapshots = [2, 4, 6, 8, 12, 16, 24, 32]
table = []
for n_sp in num_linking_snapshots:
    table_row, headings = fjlt_helper(reader, n_sp)
    table.append(table_row)

table = np.array(table)

print(headings)
print(table)

In [ ]:
jfm_table(headings,table)